# gpu ram drive

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  3 14:11:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import math
warnings.filterwarnings('ignore')

In [5]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 30 kB/s 


# dataset

In [5]:
def load_data(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) 
        train = pd.concat([train,data])
    print('finish!!')
    return train

In [8]:
train = load_data('drive/MyDrive/dacon/data_re/상추/train_input/*.csv')

finish!!


In [9]:
train.describe()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
count,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000
mean,13.500000,11.500000,3.266345,4.146259,6.189912,0.660002,4.544143,7.847381,4.034263,6.428896,2.919746,4.830358,2.527593,4.190245,5.925521,9.017202
std,8.077962,6.922371,0.157941,0.208276,0.200678,0.461405,2.840180,1.904948,4.684538,5.727701,3.868949,4.974876,3.719785,4.872833,4.557179,4.745604
min,0.000000,0.000000,1.945910,2.302585,4.110874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.750000,5.750000,3.178054,4.060443,6.042633,0.000000,0.000000,7.118826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.893058
50%,13.500000,11.500000,3.295837,4.143135,6.161207,0.693147,5.533389,8.117014,0.000000,10.052166,0.000000,0.000000,0.000000,0.000000,9.131622,11.433211
75%,20.250000,17.250000,3.367296,4.304065,6.313548,1.098612,6.646391,9.208138,9.811208,11.887402,7.310550,9.771726,7.175490,9.831508,9.812249,12.086789
max,27.000000,23.000000,3.713572,4.543295,7.090910,1.791759,7.313220,10.488743,9.812249,12.584560,9.186457,12.208056,9.131622,12.154042,10.525702,13.234633


In [50]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        #x = x.drop(['시간당총광량','일간누적총광량','일간누적총광량합'],axis=1)
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].std()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].std()
            day_light = day[day['일간누적총광량']>=0]['일간누적총광량'].std()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].std()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].std()

            day_co2 = day[day['co2관측치']>=0]['co2관측치'].std()
            day_ec = day[day['ec관측치']>=0]['ec관측치'].std()
            day_temp = day[day['내부온도관측치']>=0]['내부온도관측치'].std()
            day_moi = day[day['내부습도관측치']>=0]['내부습도관측치'].std()

            day_white2 = day[day['일간누적백색광량']>=0]['일간누적백색광량'].mean()
            day_water2 = day[day['일간누적분무량']>=0]['일간누적분무량'].mean()
            day_light2 = day[day['일간누적총광량']>=0]['일간누적총광량'].mean()
            day_blue2 = day[day['일간누적청색광량']>=0]['일간누적청색광량'].mean()
            day_red2 = day[day['일간누적적색광량']>=0]['일간누적적색광량'].mean()

            day_co22 = day[day['co2관측치']>=0]['co2관측치'].mean()
            day_ec2 = day[day['ec관측치']>=0]['ec관측치'].mean()
            day_temp2 = day[day['내부온도관측치']>=0]['내부온도관측치'].mean()
            day_moi2 = day[day['내부습도관측치']>=0]['내부습도관측치'].mean()

            time_list = day['obs_time'].unique()

            for col in col_list :
                if col in ['일간누적백색광량','일간누적청색광량','일간누적적색광량','일간누적분무량','일간누적총광량',
                            '일간누적분무량합', '일간누적백색광량합', '일간누적적색광량합', '일간누적청색광량합', '일간누적총광량합']:
                    for t in range(2,len(time_list),3):
                        time = time_list[t]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        x[col+str(time)] = value1*(i+1)

                else :
                    for t in range(0,len(time_list)-2) :
                        time = time_list[t]
                        ntime = time_list[t+1]
                        nntime = time_list[t+2]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        value2 = day[day['obs_time']==ntime][col].iloc[0]
                        value3 = day[day['obs_time']==nntime][col].iloc[0]
                        x[col+str(time)+str("~")+str(nntime)] = (value1 + value2 + value3 )

            x['day_water_std'] = (day_water)
            x['day_white_std'] = (day_white)
            x['day_red_std'] = (day_red)
            x['day_blue_std'] = (day_blue)
            x['day_light_std'] = (day_light)

            x['day_co2_std'] = (day_co2)
            x['day_ec_std'] = (day_ec)
            x['day_temp_std'] = (day_temp)
            x['day_moi_std'] = (day_moi)

            x['day_water_weight_mean'] = (day_water2*(i+1))
            x['day_white_weight_mean'] = (day_white2*(i+1))
            x['day_red_weight_mean'] = (day_red2*(i+1))
            x['day_blue_weight_mean'] = (day_blue2*(i+1))
            x['day_light_weight_mean'] = (day_light2*(i+1))

            x['day_co2_weight_mean'] = (day_co22*(i+1))
            x['day_ec_weight_mean'] = (day_ec2*(i+1))
            x['day_temp_weight_mean'] = (day_temp2*(i+1))
            x['day_moi_weight_mean'] = (day_moi2*(i+1))

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [51]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data_re/상추/train_input/*.csv', 'drive/MyDrive/dacon/data_re/상추/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data_re/상추/test_input/*.csv', 'drive/MyDrive/dacon/data_re/상추/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [56]:
train.to_csv('drive/MyDrive/dacon/data_re/상추/train_log_1204_Std.csv')
test.to_csv('drive/MyDrive/dacon/data_re/상추/test_log_1204_std.csv')

In [7]:
# train = pd.read_csv('drive/MyDrive/dacon/data_re/상추/train1203.csv').iloc[:,1:]
# test = pd.read_csv('drive/MyDrive/dacon/data_re/상추/test1203.csv').iloc[:,1:]

In [14]:
import sklearn.ensemble

In [ ]:
#  "random_state":42,
#       'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
#       'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#       "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
#       "max_depth":trial.suggest_int("max_depth", 4, 16),
#       'random_strength' :trial.suggest_int('random_strength', 0, 100),
#       "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
#       "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
#       "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#       "max_bin": trial.suggest_int("max_bin", 200, 500),
#       'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#   }

In [53]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost

X = train.drop(['predicted_weight_g'],axis=1)
y = train['predicted_weight_g']
CAT_model= {}
for f in range(0,28):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      CAT = catboost.CatBoostRegressor(task_type='GPU', verbose=0,random_state=404)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')

====================================1============================================
0 27
1 Fold MAE = 14.26804913699857


====================================2============================================
28 55
2 Fold MAE = 8.776900125778281


====================================3============================================
56 83
3 Fold MAE = 31.669099249220036


====================================4============================================
84 111
4 Fold MAE = 26.849978464218243


====================================5============================================
112 139
5 Fold MAE = 30.88850349593903


====================================6============================================
140 167
6 Fold MAE = 15.80961471186454


====================================7============================================
168 195
7 Fold MAE = 15.203599419131125


====================================8============================================
196 223
8 Fold MAE = 9.60972090331145


======================

In [54]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(28):
    pc += CAT_model[fold].predict(testX)/28
pc

array([  1.39394914,   1.61035148,   2.13673024,   2.32807216,
         3.384276  ,   3.03457355,   2.86605173,   3.42206302,
         4.09373431,   5.2020319 ,   7.70929343,  11.33744639,
        17.84782288,  23.71603768,  10.71628297,  27.98073076,
        32.77767253,  25.14534714,  22.00070554,  32.84162932,
        51.96598065,  56.99984903,  66.6890167 ,  71.92888465,
        81.67333948,  81.71012141,  88.12041243, 103.69834679,
         1.07121419,   0.72488753,   1.00424418,   0.96502896,
         1.30585121,   1.04636343,   1.99361432,   2.050021  ,
         2.93837022,   4.37377087,   5.24797752,   8.0780586 ,
        10.62940892,  10.2767736 ,  16.50862904,  15.56541091,
        18.51529549,  22.18753611,  23.57681145,  29.80035546,
        44.84962708,  43.20554426,  43.72058149,  44.21625958,
        51.72374276,  44.55807245,  57.73562112,  63.16840208,
         1.7726137 ,   2.72153016,   2.90905155,   3.03218158,
         3.5178696 ,   4.33495852,   3.51797211,   3.92

In [55]:
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data_re/상추/test_target/*.csv'))
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1